The KNN algorithm is very simple to implement, as it does not need to be trained. The training phase merely stores the training data. For each test point, we calculate the distance of that data point to every existing data point and find the $K$ closest ones. What we return is the the most common amongst the top k classification nearest to the test point. Here's the pseudocode for _K_ Nearest Neighbors:

```
kNN:

    Learn:
        Store training set T to X_train: X_train <-- T


    Predict:
        for every point xp in X_predict:
            for every point x in X_train:
                calculate the distance d in D between x and xp
            sort D in increasing order
            take the "k" items in X_train with the smallest distances to x
            return the majority class among these k items
```

Note that for large data sets, the algorithm can take very long to classify because it has to calculate the distance between the test point and evey other point in the data!


# Distance Metrics - Note

As explained above, KNN assigns a class to the test point based on the majority class of  KK  nearest neighbours. In general, euclidean distance is used to find nearest neighbours, but other distance metrics can also be used.

As the dimensionality of the feature space increases, the euclidean distance often becomes problematic due to the curse of dimensionality (discussed later).

In such cases, alternative vector-based similarity measures (dot product, cosine similarity, etc) are used to find the nearest neighbours. This transforms the original metric space into one more amicable to point-to-point measurements.

Another distance measure that you might consider is Mahalanobis distance. Here's the wikipedia page on Mahalanobis distance. https://en.wikipedia.org/wiki/Mahalanobis_distance Mahalanobis distance attempts to weight features according to their probabilities. On some data sets that may be important.

In general, it's probably a good idea to normalize the data at a minimum. Here's a link to the scikit learn scaling package. http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html . You have to be a little circumspect about employing any technique where the answers change with scaling.

In [114]:
import numpy as np
import pandas as pd

file1 = '/Users/dbuettner/nf-ds/KNN/KNN_without_solutions/nba_2013.csv'

df1 = pd.read_csv(file1)
df1.head(5)
#df1.info()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


In [104]:
len(df1)

481

In [109]:
df1.pos.nunique()

7

In [11]:
age = df1.age     # test dependent variable
pts = df1.pts     # test independent variable. feature, do higher points are scored by younger players?

pts.describe()

count     481.000000
mean      516.582121
std       470.422228
min         0.000000
25%       115.000000
50%       401.000000
75%       821.000000
max      2593.000000
Name: pts, dtype: float64

In [89]:
def distance(a, b, c=2):
    
    #dist = np.power(sum([np.abs(xi-yi)**c for xi, yi in zip(a, b)]), 1/c)
    #dist = np.power(sum([np.abs(a[i]-b[i])**c for i in range(len(a))]), 1/c)
    dist = np.power(np.abs(a-b)**c, 1/c)
    return dist

def mean(labels):
    return sum(labels) / len(labels)

def mode(labels):
    return Counter(labels).most_common(1)[0][0]

In [163]:
def knn(x_data, y_data, x_query, k, distance_fn=distance, c=2, dist_val_ind=False): #, choice_fn):
    
    knn_dist_val_ind = []
    y_query = []
    
    for i in range(len(x_query)):
        dist_and_ind = []
        for j in range(len(x_data)):
            
            #dist_and_ind.append((distance_fn(x_query[i], x_data[j], c), j))
            dist_and_ind.append((np.power(np.abs(x_query[i] - x_data[j])**c, 1/c), j))
            
        sorted_distance = sorted(dist_and_ind)        
        knn_dist_and_ind = sorted_distance[:k]
        
        knn_labels = [y_data[ind] for dist, ind in knn_dist_and_ind]
        knn_labels_mean = np.mean(knn_labels)
        
        knn_dist_val_ind.append([(dist, y_data[ind], ind) for dist, ind in knn_dist_and_ind])
        y_query.append(knn_labels_mean)
        
    if dist_val_ind == True:    
        return knn_dist_val_ind, y_query
    else:
        return y_query

In [164]:
age = df1.age     # test dependent variable
pts = df1.pts     # test independent variable. feature, do higher points are scored by younger players?

pts_query = [120, 300, 450, 600, 750, 800]

knn(pts, age, pts_query, 3, distance, 2, dist_val_ind=True)

([[(4.0, 36, 309), (5.0, 22, 70), (5.0, 34, 435)],
  [(1.0, 27, 401), (2.0, 35, 36), (2.0, 24, 269)],
  [(2.0, 20, 251), (4.0, 33, 361), (7.0, 29, 372)],
  [(3.0, 27, 466), (7.0, 22, 338), (8.0, 20, 186)],
  [(4.0, 31, 203), (8.0, 27, 443), (9.0, 27, 457)],
  [(1.0, 26, 234), (4.0, 24, 313), (9.0, 35, 282)]],
 [30.666666666666668,
  28.666666666666668,
  27.333333333333332,
  23.0,
  28.333333333333332,
  28.333333333333332])

In [107]:
knn(pts, age, pts_query, 3, distance, 2, dist_val_ind=True)

([[(4.0, 36, 309), (5.0, 22, 70), (5.0, 34, 435)],
  [(1.0, 27, 401), (2.0, 35, 36), (2.0, 24, 269)],
  [(2.0, 20, 251), (4.0, 33, 361), (7.0, 29, 372)],
  [(3.0, 27, 466), (7.0, 22, 338), (8.0, 20, 186)],
  [(4.0, 31, 203), (8.0, 27, 443), (9.0, 27, 457)],
  [(1.0, 26, 234), (4.0, 24, 313), (9.0, 35, 282)]],
 [30.666666666666668,
  28.666666666666668,
  27.333333333333332,
  23.0,
  28.333333333333332,
  28.333333333333332])

In [108]:
age_query = [20, 22, 24, 26, 28, 30, 32, 34]
knn(age, pts, age_query, 5, distance, 2)

[719.0, 508.0, 436.6, 546.4, 942.0, 483.0, 661.2, 272.4]

In [113]:
import numpy as np
import pandas as pd

file2 = '/Users/dbuettner/nf-ds/KNN/KNN_without_solutions/iris.csv'

df2 = pd.read_csv(file2)
df2.head(5)
#df2.info()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [143]:
X = df2.iloc[:, :-1]
#X = df2[:][:-2]
X.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [128]:
X.iloc[0, :]


sepal_length    5.1
sepal_width     3.5
petal_length    1.4
petal_width     0.2
Name: 0, dtype: float64

In [120]:
y = df2.iloc[:, -1]
y.head()

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
Name: species, dtype: object

In [253]:
from collections import Counter
def knn2(x_data, y_data, x_query, k, c=2, dist_val_ind=False): #, choice_fn):
    
    knn_dist_val_ind = []
    y_query = []
    
    for i in range(len(x_query)):
        dist_and_ind = []
        for j in range(len(x_data)):
            
            #dist_and_ind.append((distance_fn(x_query[i], x_data[j], c), j))
            dist_and_ind.append((np.power(sum(np.abs(x_query[i][:] - x_data.iloc[j, :])**c), 1/c), j)) # !!!
            
        sorted_distance = sorted(dist_and_ind)        
        knn_dist_and_ind = sorted_distance[:k]
        
        knn_labels = [y_data[ind] for dist, ind in knn_dist_and_ind]
        knn_labels_mode = Counter(knn_labels).most_common(1)[0][0]
        #knn_labels_mean = np.mean(knn_labels)
        
        knn_dist_val_ind.append([(dist, y_data[ind], ind) for dist, ind in knn_dist_and_ind])
        y_query.append(knn_labels_mode)
        
    if dist_val_ind == True:    
        return knn_dist_val_ind, y_query
    else:
        return y_query

In [254]:
#X_query = pd.Series([5.3, 3.0, 1.7, 0.4], index=range(4))
X_query = [[5.3, 3.0, 1.7, 0.4], [6.3, 4.0, 2.7, 1.4]]
X_query[1][:]

[6.3, 4.0, 2.7, 1.4]

In [255]:
X2 = X.iloc[:5, :]
#X2 = X[:5][:]
X2

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [256]:
dist = []
for j in range(len(X_query)):
    for i in range(len(X2)):
        #print(X_query[j][:])
        #print(X2.iloc[i, :])
        #print(X2.iloc[i, :] - X_query[j][:])
        dist.append((np.power(sum(np.abs(X2.iloc[i, :] - X_query[j][:])**2), 1/2), i, j))
sorted(dist)
#dist

[(0.53851648071345, 1, 0),
 (0.6480740698407862, 0, 0),
 (0.7615773105863909, 4, 0),
 (0.761577310586391, 3, 0),
 (0.7745966692414832, 2, 0),
 (2.195449840010015, 0, 1),
 (2.23159136044214, 4, 1),
 (2.467792535850613, 1, 1),
 (2.565151067676132, 3, 1),
 (2.5690465157330253, 2, 1)]

In [257]:
dist[3][0]

0.761577310586391

In [258]:
knn2(X, y, X_query, 5, 2, dist_val_ind=True)

([[(0.37416573867739394, 'Iris-setosa', 25),
   (0.37416573867739406, 'Iris-setosa', 23),
   (0.45825756949558405, 'Iris-setosa', 20),
   (0.45825756949558405, 'Iris-setosa', 31),
   (0.5099019513592783, 'Iris-setosa', 26)],
  [(1.5874507866387544, 'Iris-versicolor', 64),
   (1.6155494421403511, 'Iris-setosa', 18),
   (1.6792855623746663, 'Iris-setosa', 5),
   (1.7204650534085253, 'Iris-setosa', 15),
   (1.7663521732655691, 'Iris-versicolor', 79)]],
 ['Iris-setosa', 'Iris-setosa'])